In [1]:
import pandas as pd
import quandl
import math
import numpy as np
from sklearn import preprocessing, cross_validation, svm
from sklearn.linear_model import LinearRegression


D:\Anaconda\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Regression is typically used for forecast

df = quandl.get("WIKI/TSLA")

df = df[["Adj. Open", "Adj. High", "Adj. Low", "Adj. Close", "Adj. Volume"]]
df["HL_PCT"] = (df["Adj. High"] - df["Adj. Close"]) / df["Adj. Close"] * 100.0  # PCT - percent
df["PCT_change"] = (df["Adj. Close"] - df["Adj. Open"]) / df["Adj. Open"] * 100.0
df = df[["Adj. Close", "HL_PCT", "PCT_change", "Adj. Volume"]]

df.fillna(-9999, inplace=True)  # Fills the not a number with -99999, resulting an outlier
df.tail()

,Adj. Close,HL_PCT,PCT_change,Adj. Volume
Date,,,,
2017-07-05,327.09,6.160384,-5.792051,16883496.0
2017-07-06,308.89,3.852472,-2.638215,19189195.0
2017-07-07,313.22,1.206819,-0.089314,14053750.0
2017-07-10,316.05,0.598007,1.006711,13702080.0
2017-07-11,327.22,0.018336,3.550633,11400054.0


In [3]:
forecast_col = "Adj. Close"
forecast_out = int(math.ceil(0.01 * len(df)))  # 1% of the frame.

df["label"] = df[forecast_col].shift(-forecast_out)
df.dropna(inplace=True)

df.tail()

,Adj. Close,HL_PCT,PCT_change,Adj. Volume,label
Date,,,,,
2017-06-08,370.00,0.513514,1.718213,8975028.0,327.09
2017-06-09,357.32,5.471286,-4.567064,17160231.0,308.89
2017-06-12,359.01,1.529205,0.284924,10476833.0,313.22
2017-06-13,375.95,0.013300,2.265927,11693404.0,316.05
2017-06-14,380.66,0.943099,-0.111524,12731997.0,327.22


In [4]:
X = np.array(df.drop(["label"], 1))
y = np.array(df["label"])

X = preprocessing.scale(X)  # Adds processing time
y = np.array(df["label"])

In [5]:
# Training
